In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import geopandas as gpd
import folium
import contextily as cx
from shapely.geometry import Point, LineString, Polygon
import numpy as np
from scipy.spatial import cKDTree
from geopy.distance import distance
import scipy.stats as stats


/Users/jnapolitano/venvs/finance/lib/python3.9/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


## Shape Data

In [2]:
gisfilepath = "/Users/jnapolitano/Projects/wattime-takehome/data/country_shapefiles/World_Countries__Generalized_.shp"


countries_df = gpd.read_file(gisfilepath)

countries_df = countries_df.to_crs(epsg=3857)

In [3]:
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))

cities = gpd.read_file(gpd.datasets.get_path('naturalearth_cities'))

In [4]:
world.rename({"iso_a3" :  'iso3_country'},axis =1,inplace=True)

In [5]:
world

,pop_est,continent,name,iso3_country,gdp_md_est,geometry
0,920938,Oceania,Fiji,FJI,8374.0,"MULTIPOLYGON (((180.00000 -16.06713, 180.00000..."
1,53950935,Africa,Tanzania,TZA,150600.0,"POLYGON ((33.90371 -0.95000, 34.07262 -1.05982..."
2,603253,Africa,W. Sahara,ESH,906.5,"POLYGON ((-8.66559 27.65643, -8.66512 27.58948..."
3,35623680,North America,Canada,CAN,1674000.0,"MULTIPOLYGON (((-122.84000 49.00000, -122.9742..."
4,326625791,North America,United States of America,USA,18560000.0,"MULTIPOLYGON (((-122.84000 49.00000, -120.0000..."
...,...,...,...,...,...,...
172,7111024,Europe,Serbia,SRB,101800.0,"POLYGON ((18.82982 45.90887, 18.82984 45.90888..."
173,642550,Europe,Montenegro,MNE,10610.0,"POLYGON ((20.07070 42.58863, 19.80161 42.50009..."
174,1895250,Europe,Kosovo,-99,18490.0,"POLYGON ((20.59025 41.85541, 20.52295 42.21787..."
175,1218208,North America,Trinidad and Tobago,TTO,43570.0,"POLYGON ((-61.68000 10.76000, -61.10500 10.890..."


## FAOSTAT DATA

In [6]:
filepath = "/Users/jnapolitano/Projects/wattime-takehome/data/FAOSTAT_DATA.csv"

faostat_emissions_df = pd.read_csv(filepath)
faostat_emissions_df.drop('Unnamed: 0', axis = 1 , inplace = True)
faostat_emissions_df = faostat_emissions_df[(faostat_emissions_df["country_name"] != 'Total') & (faostat_emissions_df["country_name"] != "mean")]
faostat_emissions_df.dropna(how="any", axis=1, inplace= True)

In [7]:
faostat_emissions_df

,iso3_country,country_name,2015,2016,2017,2018,2019,tCO2_2015,tCO2_2016,tCO2_2017,tCO2_2018,tCO2_2019,Mean_CH4,Total_CH4,Mean_CO2,Total_CO2
0,BGD,Bangladesh,1131293.4,1093480.4,1154531.0,1144591.0,1144745.4,28282335.0,27337010.0,28863275.0,28614775.0,28618635.0,1133728.24,5668641.2,28343206.0,141716030.0
1,BRA,Brazil,138910.3,126278.2,130322.9,121615.2,111084.8,3472757.5,3156955.0,3258072.5,3040380.0,2777120.0,125642.28,628211.4,3141057.0,15705285.0
2,CHN,China,5406593.9,5399920.0,5400129.0,5302173.1,5214454.7,135164847.5,134998000.0,135003225.0,132554327.5,130361367.5,5344654.14,26723270.7,133616353.5,668081767.5
3,ESP,Spain,55082.2,55073.1,54232.4,52925.0,52098.5,1377055.0,1376827.5,1355810.0,1323125.0,1302462.5,53882.24,269411.2,1347056.0,6735280.0
4,IDN,Indonesia,2407953.5,2387656.4,2425290.6,2405613.8,2257604.3,60198837.5,59691410.0,60632265.0,60140345.0,56440107.5,2376823.72,11884118.6,59420593.0,297102965.0
5,IND,India,4580248.4,4559136.4,4620790.8,4661154.9,4621416.8,114506210.0,113978410.0,115519770.0,116528872.5,115535420.0,4608549.46,23042747.3,115213736.5,576068682.5
6,IRN,Iran (Islamic Republic of),116486.7,131008.5,87233.6,93936.6,96103.4,2912167.5,3275212.5,2180840.0,2348415.0,2402585.0,104953.76,524768.8,2623844.0,13119220.0
7,ITA,Italy,114574.8,118003.0,118003.0,109463.8,110895.1,2864370.0,2950075.0,2950075.0,2736595.0,2772377.5,114187.94,570939.7,2854698.5,14273492.5
8,JPN,Japan,330353.1,326403.0,323700.3,322245.0,320581.8,8258827.5,8160075.0,8092507.5,8056125.0,8014545.0,324656.64,1623283.2,8116416.0,40582080.0
9,KHM,Cambodia,436826.0,459003.1,473745.3,479362.7,468378.9,10920650.0,11475077.5,11843632.5,11984067.5,11709472.5,463463.20,2317316.0,11586580.0,57932900.0


### FAOSTAT Country Merge

In [8]:
faostat_merged_df = world.merge(faostat_emissions_df, on='iso3_country', how='right', sort=True)


In [9]:
faostat_merged_df
#trace_merged_df.dropna(axis=1, how='any')

,pop_est,continent,name,iso3_country,gdp_md_est,geometry,country_name,2015,2016,2017,...,2019,tCO2_2015,tCO2_2016,tCO2_2017,tCO2_2018,tCO2_2019,Mean_CH4,Total_CH4,Mean_CO2,Total_CO2
0,157826578,Asia,Bangladesh,BGD,628400.0,"POLYGON ((92.67272 22.04124, 92.65226 21.32405...",Bangladesh,1131293.4,1093480.4,1154531.0,...,1144745.4,28282335.0,27337010.0,28863275.0,28614775.0,28618635.0,1133728.24,5668641.2,28343206.0,141716030.0
1,207353391,South America,Brazil,BRA,3081000.0,"POLYGON ((-53.37366 -33.76838, -53.65054 -33.2...",Brazil,138910.3,126278.2,130322.9,...,111084.8,3472757.5,3156955.0,3258072.5,3040380.0,2777120.0,125642.28,628211.4,3141057.0,15705285.0
2,1379302771,Asia,China,CHN,21140000.0,"MULTIPOLYGON (((109.47521 18.19770, 108.65521 ...",China,5406593.9,5399920.0,5400129.0,...,5214454.7,135164847.5,134998000.0,135003225.0,132554327.5,130361367.5,5344654.14,26723270.7,133616353.5,668081767.5
3,48958159,Europe,Spain,ESP,1690000.0,"POLYGON ((-7.45373 37.09779, -7.53711 37.42890...",Spain,55082.2,55073.1,54232.4,...,52098.5,1377055.0,1376827.5,1355810.0,1323125.0,1302462.5,53882.24,269411.2,1347056.0,6735280.0
4,260580739,Asia,Indonesia,IDN,3028000.0,"MULTIPOLYGON (((141.00021 -2.60015, 141.01706 ...",Indonesia,2407953.5,2387656.4,2425290.6,...,2257604.3,60198837.5,59691410.0,60632265.0,60140345.0,56440107.5,2376823.72,11884118.6,59420593.0,297102965.0
5,1281935911,Asia,India,IND,8721000.0,"POLYGON ((97.32711 28.26158, 97.40256 27.88254...",India,4580248.4,4559136.4,4620790.8,...,4621416.8,114506210.0,113978410.0,115519770.0,116528872.5,115535420.0,4608549.46,23042747.3,115213736.5,576068682.5
6,82021564,Asia,Iran,IRN,1459000.0,"POLYGON ((48.56797 29.92678, 48.01457 30.45246...",Iran (Islamic Republic of),116486.7,131008.5,87233.6,...,96103.4,2912167.5,3275212.5,2180840.0,2348415.0,2402585.0,104953.76,524768.8,2623844.0,13119220.0
7,62137802,Europe,Italy,ITA,2221000.0,"MULTIPOLYGON (((10.44270 46.89355, 11.04856 46...",Italy,114574.8,118003.0,118003.0,...,110895.1,2864370.0,2950075.0,2950075.0,2736595.0,2772377.5,114187.94,570939.7,2854698.5,14273492.5
8,126451398,Asia,Japan,JPN,4932000.0,"MULTIPOLYGON (((141.88460 39.18086, 140.95949 ...",Japan,330353.1,326403.0,323700.3,...,320581.8,8258827.5,8160075.0,8092507.5,8056125.0,8014545.0,324656.64,1623283.2,8116416.0,40582080.0
9,16204486,Asia,Cambodia,KHM,58940.0,"POLYGON ((102.58493 12.18659, 102.34810 13.394...",Cambodia,436826.0,459003.1,473745.3,...,468378.9,10920650.0,11475077.5,11843632.5,11984067.5,11709472.5,463463.20,2317316.0,11586580.0,57932900.0


In [10]:
faostat_merged_df = faostat_merged_df[faostat_merged_df['continent'] != 'Antarctica'].copy()

### FAOSTAT Merge to File

In [11]:
gisout = "/Users/jnapolitano/Projects/wattime-takehome/data/FAOSTAT_GEO.geojson"
faostat_merged_df.to_file(gisout, driver="GeoJSON")

/Users/jnapolitano/venvs/finance/lib/python3.9/site-packages/geopandas/io/file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,


## TRACE Data

In [12]:
filepath = "/Users/jnapolitano/Projects/wattime-takehome/data/TRACE_DATA.csv"

trace_emissions_df = pd.read_csv(filepath)
trace_emissions_df.drop('Unnamed: 0', axis = 1 , inplace = True)
trace_emissions_df = trace_emissions_df[(trace_emissions_df["country_name"] != 'Total') & (trace_emissions_df["country_name"] != "mean")]

In [13]:
trace_emissions_df

,iso3_country,country_name,tCH4_2015,tCH4_2016,tCH4_2017,tCH4_2018,tCH4_2019,tCH4_2020,tCH4_2021,tCO2_2015,tCO2_2016,tCO2_2017,tCO2_2018,tCO2_2019,Mean_CH4,Total_CH4,Mean_CO2,Total_CO2
0,BGD,Bangladesh,2.344420e+06,2.278158e+06,2.098958e+06,2.141231e+06,2.070985e+06,2.106781e+06,1.983974e+06,5.861049e+07,5.695395e+07,5.247394e+07,5.353076e+07,5.177463e+07,2.186750e+06,1.093375e+07,5.466875e+07,2.733438e+08
1,BRA,Brazil,3.410233e+05,3.104189e+05,3.725173e+05,3.717030e+05,3.294713e+05,4.902874e+05,4.544874e+05,8.525583e+06,7.760473e+06,9.312934e+06,9.292575e+06,8.236783e+06,3.450268e+05,1.725134e+06,8.625670e+06,4.312835e+07
2,CHN,China,6.133647e+06,5.859531e+06,6.355071e+06,5.413962e+06,5.603352e+06,6.402353e+06,6.068210e+06,1.533412e+08,1.464883e+08,1.588768e+08,1.353491e+08,1.400838e+08,5.873113e+06,2.936556e+07,1.468278e+08,7.341391e+08
3,ESP,Spain,1.141464e+04,1.334803e+04,1.217299e+04,1.405410e+04,1.148324e+04,1.305461e+04,8.531579e+03,2.853661e+05,3.337007e+05,3.043248e+05,3.513524e+05,2.870810e+05,1.249460e+04,6.247300e+04,3.123650e+05,1.561825e+06
4,IDN,Indonesia,1.283649e+06,1.023129e+06,9.615327e+05,1.176982e+06,1.266668e+06,1.188195e+06,1.009936e+06,3.209122e+07,2.557824e+07,2.403832e+07,2.942454e+07,3.166670e+07,1.142392e+06,5.711960e+06,2.855980e+07,1.427990e+08
5,IND,India,6.219887e+06,5.309413e+06,6.228451e+06,6.589798e+06,7.501556e+06,7.599764e+06,6.567960e+06,1.554972e+08,1.327353e+08,1.557113e+08,1.647450e+08,1.875389e+08,6.369821e+06,3.184910e+07,1.592455e+08,7.962276e+08
6,IRN,Iran (Islamic Republic of),8.774407e+04,9.180121e+04,9.620217e+04,8.875744e+04,9.500199e+04,9.600254e+04,9.053525e+04,2.193602e+06,2.295030e+06,2.405054e+06,2.218936e+06,2.375050e+06,9.190138e+04,4.595069e+05,2.297534e+06,1.148767e+07
7,ITA,Italy,4.995968e+04,4.937785e+04,5.443679e+04,4.469902e+04,4.566914e+04,5.101547e+04,5.089759e+04,1.248992e+06,1.234446e+06,1.360920e+06,1.117475e+06,1.141729e+06,4.882850e+04,2.441425e+05,1.220712e+06,6.103562e+06
8,JPN,Japan,2.305465e+05,2.284133e+05,2.708935e+05,1.548252e+05,2.332056e+05,2.835167e+05,1.574007e+05,5.763662e+06,5.710333e+06,6.772337e+06,3.870631e+06,5.830141e+06,2.235768e+05,1.117884e+06,5.589421e+06,2.794710e+07
9,KHM,Cambodia,4.954698e+05,5.731698e+05,4.517045e+05,5.592610e+05,5.947277e+05,6.412802e+05,5.644891e+05,1.238675e+07,1.432925e+07,1.129261e+07,1.398153e+07,1.486819e+07,5.348666e+05,2.674333e+06,1.337166e+07,6.685832e+07


## TRACE Country Merge

In [14]:
trace_merged_df = world.merge(trace_emissions_df, on='iso3_country', how='right', sort=True)

In [15]:
trace_merged_df

,pop_est,continent,name,iso3_country,gdp_md_est,geometry,country_name,tCH4_2015,tCH4_2016,tCH4_2017,...,tCH4_2021,tCO2_2015,tCO2_2016,tCO2_2017,tCO2_2018,tCO2_2019,Mean_CH4,Total_CH4,Mean_CO2,Total_CO2
0,157826578,Asia,Bangladesh,BGD,628400.0,"POLYGON ((92.67272 22.04124, 92.65226 21.32405...",Bangladesh,2.344420e+06,2.278158e+06,2.098958e+06,...,1.983974e+06,5.861049e+07,5.695395e+07,5.247394e+07,5.353076e+07,5.177463e+07,2.186750e+06,1.093375e+07,5.466875e+07,2.733438e+08
1,207353391,South America,Brazil,BRA,3081000.0,"POLYGON ((-53.37366 -33.76838, -53.65054 -33.2...",Brazil,3.410233e+05,3.104189e+05,3.725173e+05,...,4.544874e+05,8.525583e+06,7.760473e+06,9.312934e+06,9.292575e+06,8.236783e+06,3.450268e+05,1.725134e+06,8.625670e+06,4.312835e+07
2,1379302771,Asia,China,CHN,21140000.0,"MULTIPOLYGON (((109.47521 18.19770, 108.65521 ...",China,6.133647e+06,5.859531e+06,6.355071e+06,...,6.068210e+06,1.533412e+08,1.464883e+08,1.588768e+08,1.353491e+08,1.400838e+08,5.873113e+06,2.936556e+07,1.468278e+08,7.341391e+08
3,48958159,Europe,Spain,ESP,1690000.0,"POLYGON ((-7.45373 37.09779, -7.53711 37.42890...",Spain,1.141464e+04,1.334803e+04,1.217299e+04,...,8.531579e+03,2.853661e+05,3.337007e+05,3.043248e+05,3.513524e+05,2.870810e+05,1.249460e+04,6.247300e+04,3.123650e+05,1.561825e+06
4,260580739,Asia,Indonesia,IDN,3028000.0,"MULTIPOLYGON (((141.00021 -2.60015, 141.01706 ...",Indonesia,1.283649e+06,1.023129e+06,9.615327e+05,...,1.009936e+06,3.209122e+07,2.557824e+07,2.403832e+07,2.942454e+07,3.166670e+07,1.142392e+06,5.711960e+06,2.855980e+07,1.427990e+08
5,1281935911,Asia,India,IND,8721000.0,"POLYGON ((97.32711 28.26158, 97.40256 27.88254...",India,6.219887e+06,5.309413e+06,6.228451e+06,...,6.567960e+06,1.554972e+08,1.327353e+08,1.557113e+08,1.647450e+08,1.875389e+08,6.369821e+06,3.184910e+07,1.592455e+08,7.962276e+08
6,82021564,Asia,Iran,IRN,1459000.0,"POLYGON ((48.56797 29.92678, 48.01457 30.45246...",Iran (Islamic Republic of),8.774407e+04,9.180121e+04,9.620217e+04,...,9.053525e+04,2.193602e+06,2.295030e+06,2.405054e+06,2.218936e+06,2.375050e+06,9.190138e+04,4.595069e+05,2.297534e+06,1.148767e+07
7,62137802,Europe,Italy,ITA,2221000.0,"MULTIPOLYGON (((10.44270 46.89355, 11.04856 46...",Italy,4.995968e+04,4.937785e+04,5.443679e+04,...,5.089759e+04,1.248992e+06,1.234446e+06,1.360920e+06,1.117475e+06,1.141729e+06,4.882850e+04,2.441425e+05,1.220712e+06,6.103562e+06
8,126451398,Asia,Japan,JPN,4932000.0,"MULTIPOLYGON (((141.88460 39.18086, 140.95949 ...",Japan,2.305465e+05,2.284133e+05,2.708935e+05,...,1.574007e+05,5.763662e+06,5.710333e+06,6.772337e+06,3.870631e+06,5.830141e+06,2.235768e+05,1.117884e+06,5.589421e+06,2.794710e+07
9,16204486,Asia,Cambodia,KHM,58940.0,"POLYGON ((102.58493 12.18659, 102.34810 13.394...",Cambodia,4.954698e+05,5.731698e+05,4.517045e+05,...,5.644891e+05,1.238675e+07,1.432925e+07,1.129261e+07,1.398153e+07,1.486819e+07,5.348666e+05,2.674333e+06,1.337166e+07,6.685832e+07


In [16]:
trace_merged_df = trace_merged_df[trace_merged_df['continent'] != 'Antarctica'].copy()

In [17]:
trace_merged_df
#trace_merged_df.dropna(axis=1, how='any')

,pop_est,continent,name,iso3_country,gdp_md_est,geometry,country_name,tCH4_2015,tCH4_2016,tCH4_2017,...,tCH4_2021,tCO2_2015,tCO2_2016,tCO2_2017,tCO2_2018,tCO2_2019,Mean_CH4,Total_CH4,Mean_CO2,Total_CO2
0,157826578,Asia,Bangladesh,BGD,628400.0,"POLYGON ((92.67272 22.04124, 92.65226 21.32405...",Bangladesh,2.344420e+06,2.278158e+06,2.098958e+06,...,1.983974e+06,5.861049e+07,5.695395e+07,5.247394e+07,5.353076e+07,5.177463e+07,2.186750e+06,1.093375e+07,5.466875e+07,2.733438e+08
1,207353391,South America,Brazil,BRA,3081000.0,"POLYGON ((-53.37366 -33.76838, -53.65054 -33.2...",Brazil,3.410233e+05,3.104189e+05,3.725173e+05,...,4.544874e+05,8.525583e+06,7.760473e+06,9.312934e+06,9.292575e+06,8.236783e+06,3.450268e+05,1.725134e+06,8.625670e+06,4.312835e+07
2,1379302771,Asia,China,CHN,21140000.0,"MULTIPOLYGON (((109.47521 18.19770, 108.65521 ...",China,6.133647e+06,5.859531e+06,6.355071e+06,...,6.068210e+06,1.533412e+08,1.464883e+08,1.588768e+08,1.353491e+08,1.400838e+08,5.873113e+06,2.936556e+07,1.468278e+08,7.341391e+08
3,48958159,Europe,Spain,ESP,1690000.0,"POLYGON ((-7.45373 37.09779, -7.53711 37.42890...",Spain,1.141464e+04,1.334803e+04,1.217299e+04,...,8.531579e+03,2.853661e+05,3.337007e+05,3.043248e+05,3.513524e+05,2.870810e+05,1.249460e+04,6.247300e+04,3.123650e+05,1.561825e+06
4,260580739,Asia,Indonesia,IDN,3028000.0,"MULTIPOLYGON (((141.00021 -2.60015, 141.01706 ...",Indonesia,1.283649e+06,1.023129e+06,9.615327e+05,...,1.009936e+06,3.209122e+07,2.557824e+07,2.403832e+07,2.942454e+07,3.166670e+07,1.142392e+06,5.711960e+06,2.855980e+07,1.427990e+08
5,1281935911,Asia,India,IND,8721000.0,"POLYGON ((97.32711 28.26158, 97.40256 27.88254...",India,6.219887e+06,5.309413e+06,6.228451e+06,...,6.567960e+06,1.554972e+08,1.327353e+08,1.557113e+08,1.647450e+08,1.875389e+08,6.369821e+06,3.184910e+07,1.592455e+08,7.962276e+08
6,82021564,Asia,Iran,IRN,1459000.0,"POLYGON ((48.56797 29.92678, 48.01457 30.45246...",Iran (Islamic Republic of),8.774407e+04,9.180121e+04,9.620217e+04,...,9.053525e+04,2.193602e+06,2.295030e+06,2.405054e+06,2.218936e+06,2.375050e+06,9.190138e+04,4.595069e+05,2.297534e+06,1.148767e+07
7,62137802,Europe,Italy,ITA,2221000.0,"MULTIPOLYGON (((10.44270 46.89355, 11.04856 46...",Italy,4.995968e+04,4.937785e+04,5.443679e+04,...,5.089759e+04,1.248992e+06,1.234446e+06,1.360920e+06,1.117475e+06,1.141729e+06,4.882850e+04,2.441425e+05,1.220712e+06,6.103562e+06
8,126451398,Asia,Japan,JPN,4932000.0,"MULTIPOLYGON (((141.88460 39.18086, 140.95949 ...",Japan,2.305465e+05,2.284133e+05,2.708935e+05,...,1.574007e+05,5.763662e+06,5.710333e+06,6.772337e+06,3.870631e+06,5.830141e+06,2.235768e+05,1.117884e+06,5.589421e+06,2.794710e+07
9,16204486,Asia,Cambodia,KHM,58940.0,"POLYGON ((102.58493 12.18659, 102.34810 13.394...",Cambodia,4.954698e+05,5.731698e+05,4.517045e+05,...,5.644891e+05,1.238675e+07,1.432925e+07,1.129261e+07,1.398153e+07,1.486819e+07,5.348666e+05,2.674333e+06,1.337166e+07,6.685832e+07


### Trace Merge to File

In [18]:
gisout = "/Users/jnapolitano/Projects/wattime-takehome/data/TRACE_DATA_GEO.geojson"
trace_merged_df.to_file(gisout, driver="GeoJSON")

/Users/jnapolitano/venvs/finance/lib/python3.9/site-packages/geopandas/io/file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,


## Merge GEO DATA

In [19]:
filepath = "/Users/jnapolitano/Projects/wattime-takehome/data/MERGED_DATA.csv"

merged_data = pd.read_csv(filepath)

merged_data.drop('Unnamed: 0', axis = 1 , inplace = True)


In [20]:
merged_countries = world.merge(merged_data, on='iso3_country', how='right', sort=True)

In [21]:
merged_countries

,pop_est,continent,name,iso3_country,gdp_md_est,geometry,country_name_FAOSTAT,2015,2016,2017,...,CO2_abs_percent_diff_2019,CO2_abs_percent_diff_means,CO2_abs_percent_diff_totals,CO2_relative_percent_diff_2015,CO2_relative_percent_diff_2016,CO2_relative_percent_diff_2017,CO2_relative_percent_diff_2018,CO2_relative_percent_diff_2019,CO2_relative_percent_diff_means,CO2_relative_percent_diff_totals
0,1.578266e+08,Asia,Bangladesh,BGD,628400.0,"POLYGON ((92.67272 22.04124, 92.65226 21.32405...",Bangladesh,1131293.4,1.093480e+06,1.154531e+06,...,57.606797,63.425917,63.425917,-107.233570,-108.340089,-81.801753,-87.073857,-80.912284,-92.881337,-92.881337
1,2.073534e+08,South America,Brazil,BRA,3081000.0,"POLYGON ((-53.37366 -33.76838, -53.65054 -33.2...",Brazil,138910.3,1.262782e+05,1.303229e+05,...,99.141295,93.222402,93.222402,-145.498958,-145.821472,-185.841820,-205.638590,-196.594424,-174.610414,-174.610414
2,1.379303e+09,Asia,China,CHN,21140000.0,"MULTIPOLYGON (((109.47521 18.19770, 108.65521 ...",China,5406593.9,5.399920e+06,5.400129e+06,...,7.189945,9.421813,9.421813,-13.447527,-8.511442,-17.683689,-2.108366,-7.458060,-9.887609,-9.887609
3,4.895816e+07,Europe,Spain,ESP,1690000.0,"POLYGON ((-7.45373 37.09779, -7.53711 37.42890...",Spain,55082.2,5.507310e+04,5.423240e+04,...,127.757634,124.705066,124.705066,79.277072,75.763074,77.554025,73.445258,77.958601,76.811284,76.811284
4,2.605807e+08,Asia,Indonesia,IDN,3028000.0,"MULTIPOLYGON (((141.00021 -2.60015, 141.01706 ...",Indonesia,2407953.5,2.387656e+06,2.425291e+06,...,56.234955,70.153790,70.153790,46.691296,57.149216,60.353916,51.073546,43.893273,51.936188,51.936188
5,1.281936e+09,Asia,India,IND,8721000.0,"POLYGON ((97.32711 28.26158, 97.40256 27.88254...",India,4580248.4,4.559136e+06,4.620791e+06,...,47.515394,32.086209,32.086209,-35.798018,-16.456550,-34.791882,-41.376939,-62.321563,-38.217479,-38.217479
6,8.202156e+07,Asia,Iran,IRN,1459000.0,"POLYGON ((48.56797 29.92678, 48.01457 30.45246...",Iran (Islamic Republic of),116486.7,1.310085e+05,8.723360e+04,...,1.152675,13.260902,13.260902,24.674600,29.927285,-10.281098,5.513463,1.146070,12.436318,12.436318
7,6.213780e+07,Europe,Italy,ITA,2221000.0,"MULTIPOLYGON (((10.44270 46.89355, 11.04856 46...",Italy,114574.8,1.180030e+05,1.180030e+05,...,83.321649,80.187551,80.187551,56.395576,58.155426,53.868303,59.165480,58.817707,57.238482,57.238482
8,1.264514e+08,Asia,Japan,JPN,4932000.0,"MULTIPOLYGON (((141.88460 39.18086, 140.95949 ...",Japan,330353.1,3.264030e+05,3.237003e+05,...,31.555853,36.874729,36.874729,30.212099,30.021068,16.313490,51.954183,27.255501,31.134372,31.134372
9,1.620449e+07,Asia,Cambodia,KHM,58940.0,"POLYGON ((102.58493 12.18659, 102.34810 13.394...",Cambodia,436826.0,4.590031e+05,4.737453e+05,...,23.769728,14.304565,14.304565,-13.424988,-24.872761,4.652461,-16.667610,-26.975760,-15.406481,-15.406481


### Merged Countries to File

In [22]:
gisout = "/Users/jnapolitano/Projects/wattime-takehome/data/MERGE_DATA_GEO.geojson"
merged_countries.to_file(gisout, driver="GeoJSON")

/Users/jnapolitano/venvs/finance/lib/python3.9/site-packages/geopandas/io/file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,
